El objetivo de este notebook es mostrar cómo construir un modelo de regresión lineal y su evaluación con las métricas seleccionadas. Se explicará, paso a paso, como:
1. Preparar los datos para el modelado.
2. Construir el modelo de regresión lineal.
3. Evaluar el modelo.
4. Interpretar el modelo.

Se utiliza un conjunto de datos drelacionado con ventas de vehículos.


### 1. Importación de librerías 

En las siguientes líneas de código se importan las librerías y herramientas necesarias para desarrollar el caso de uso.

In [1]:
# Librería para comando de sistema
import os
# Librerías para manejo de datos
import pandas as pd
import numpy as np
# Librerías de aprendizaje automático.
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para construir un modelo con el algoritmo de regresión lineal
from sklearn.linear_model import LinearRegression
# Para determinar el rendimiento del modelo con las métricas MSE, MAE y R2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Para sacar un reporte estadístico que podemos usar para determinar las importancia de las variables explicativas.
import statsmodels.api as sm 

### 2. Carga de los datos
A través de la librería **pandas** podemos realizar la carga de datos desde diferentes fuentes de información, en este caso se realizará la carga de un archivo plano csv (archivo separado por comas).

In [3]:
# Se cargan los datos. 
data=pd.read_csv('Datos_venta_vehiculos.csv',sep=';')

In [4]:
# Cantidad de datos y número de variables
data.shape

(8128, 9)

In [5]:
# Mostrar los datos
data.head()

,referencia,modelo,precio,kilometraje,combustible,propietario,motor,poder_maximo,asientos
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,First Owner,1248.0,74.00,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Second Owner,1498.0,103.52,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Third Owner,1497.0,78.00,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,First Owner,1396.0,90.00,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,First Owner,1298.0,NaN,5.0


### 3. Preparación de los datos

Antes de realizar cualquier paso de preparación de datos es importante asegurar que estos no contienen errores, como datos faltantes (missing) o duplicados. La exploración realizada en el notebook del módulo 1 evidenció que algunos registros no tienen valores para algunas variables. Es necesario tratar estas ausencias ya que, en general, los algoritmos de aprendizaje no son robustos ante ellas. 

In [6]:
# Es recomendable que todos los pasos preparación se realicen sobre otro archivo.
data_t = data.copy()

In [7]:
# Podemos comprobar las ausencias con esta línea de código. 
data_t.isna().sum()/len(data_t)

referencia      0.000000
modelo          0.000000
precio          0.000000
kilometraje     0.000000
combustible     0.000000
propietario     0.000000
motor           0.027190
poder_maximo    0.027436
asientos        0.027190
dtype: float64

Aproximadamante un 2% del total de los datos tienen ausencias. Dado el bajo porcentaje se puede realizar la eliminación de estos registros.

In [8]:
# Eliminación de registros con ausencias. 
data_t=data_t.dropna()

In [9]:
# Podemos ver el tamaño del conjunto de datos después de este paso de limpieza de datos.
data_t.shape

(7905, 9)

También la exploración mostró que existen registros duplicados. Es conveniente también eliminar estos del conjunto de datos. 

In [10]:
# Eliminación de registros duplicados.
data_t=data_t.drop_duplicates()

In [11]:
# Podemos ver el tamaño del conjunto de datos después de este paso de limpieza de datos.
data_t.shape

(6716, 9)

Un aspecto muy importante para tener en cuenta son los requerimientos de entrada de los algoritmos de aprendizaje. Cada uno de estos puede trabajar con un tipo de variable. Por ejemplo, para el algoritmo de regresión lineal todas las variables en el conjunto de datos deben ser numéricas. ¿Que pasa si en el conjunto de datos hay variables categóricas? Pues que se debe realizar una transformación de estas variables a un formato numérico.

Esta tranformación se puede realizar de dos formas: asignado un número a cada categoría o representado cada categoría como una variable binaria. Esta útima transformación se conoce como 1-de-n y construye lo que se conoce como variables dummies. En general, es la más recomendada ya que no genera una relación de orden entre las categorías.

Esta numerización es la que se aplica a la variable "nombre_categoria_producto" la cual, a su vez, nos permitirá establecer la influencia de cada categoría en la variable objetivo.

In [12]:
# Se muestran las categorías de la variable "propietario" con sus frecuencias
pd.value_counts(data['propietario'])

First Owner             5289
Second Owner            2105
Third Owner              555
Fourth & Above Owner     174
Test Drive Car             5
Name: propietario, dtype: int64

In [13]:
# Se muestran las categorías de la variable "combustible" con sus frecuencias
pd.value_counts(data['combustible'])

Diesel    4402
Petrol    3631
CNG         57
LPG         38
Name: combustible, dtype: int64

In [14]:
# Se realiza la transformación de estas variables a dummies.
data_t = pd.get_dummies(data_t, columns=['combustible','propietario'])

In [15]:
# Cantidad de datos y número de variables después de esta transformación.
data_t.shape

(6716, 16)

In [16]:
# Ahora el conjunto de datos tiene 14 variables.Veamos las primeras filas de este nuevo conjunto de datos.
data_t.head()

,referencia,modelo,precio,kilometraje,motor,poder_maximo,asientos,combustible_CNG,combustible_Diesel,combustible_LPG,combustible_Petrol,propietario_First Owner,propietario_Fourth & Above Owner,propietario_Second Owner,propietario_Test Drive Car,propietario_Third Owner
0,Maruti Swift Dzire VDI,2014,450000,145500,1248.0,74.00,5.0,0,1,0,0,1,0,0,0,0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,1498.0,103.52,5.0,0,1,0,0,0,0,1,0,0
2,Honda City 2017-2020 EXi,2006,158000,140000,1497.0,78.00,5.0,0,0,0,1,0,0,0,0,1
3,Hyundai i20 Sportz Diesel,2010,225000,127000,1396.0,90.00,5.0,0,1,0,0,1,0,0,0,0
5,Hyundai Xcent 1.2 VTVT E Plus,2017,440000,45000,1197.0,81.86,50.0,0,0,0,1,1,0,0,0,0


In [17]:
data_t.to_csv("precio.csv")

Otro aspecto a que debemos considerar es la eliminación de variables que no aportan a la solución. Por ejemplo, identificadores como "referencia" no son informativos. Es importante que las eliminemos del conjunto de datos ya que pueden afectar el rendimiento de los modelos generados.

In [18]:
# Eliminación del indentoficador de fila.
data_t=data_t.drop(['referencia'], axis=1)

Con los datos preparados ya podemos construir nuestro modelo de regresión lineal.

### 4. Construcción del modelo

Los algoritmos supervisados implementados en scikit-learn requieren que las variables de entrada estén separadas de la variable objetivo. En este caso, nuestra variable objetivo es el precio.

In [31]:
# Se selecciona la variable objetivo, en este caso "precio".
Y=data_t['precio']
# Del conjunto de datos se elimina la variable "precio".
X=data_t.drop(['precio'], axis=1)

In [32]:
# Mostramos nuestros datos
X.head()

,modelo,kilometraje,motor,poder_maximo,asientos,combustible_CNG,combustible_Diesel,combustible_LPG,combustible_Petrol,propietario_First Owner,propietario_Fourth & Above Owner,propietario_Second Owner,propietario_Test Drive Car,propietario_Third Owner
0,2014,145500,1248.0,74.00,5.0,0,1,0,0,1,0,0,0,0
1,2014,120000,1498.0,103.52,5.0,0,1,0,0,0,0,1,0,0
2,2006,140000,1497.0,78.00,5.0,0,0,0,1,0,0,0,0,1
3,2010,127000,1396.0,90.00,5.0,0,1,0,0,1,0,0,0,0
5,2017,45000,1197.0,81.86,50.0,0,0,0,1,1,0,0,0,0


In [21]:
Y.head()

0    450000
1    370000
2    158000
3    225000
5    440000
Name: precio, dtype: int64

A continuación debemos hacer la separación de nuestros datos en un conjunto para el entrenamiento y otro para el test. Recuerda que esta división nos permitirá determinar el rendimiento de generalización del modelo sobre nuevos datos.

El conjunto de entrenamiento se utiliza para ajustar (entrenar) un modelo. Luego, se utiliza el conjunto test para hacer predicciones, las cuales se comparan con los valores esperados para determinar su rendimiento utilizando la métrica seleccionada. 

1. **Train data:** se utiliza para entrenar el modelo con el algoritmo de aprendizaje.
2. **Test data:** se utiliza para evaluar el ajuste del modelo.

In [33]:
# Se realiza la división entrenamiento - test. Se deja 20% de los datos para el test.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

Antes de construir el modelo debemos crear un objeto de la clase LinearRegression.

In [34]:
# Primero se crea el objeto para construir el modelo
modelo_regresion = LinearRegression()
# Podemos verificar que lo hemos construido.
modelo_regresion

LinearRegression()

A continuación, procedemos a entrenar el modelo utilizando el conjunto de entrenamiento.

In [35]:
# Ajustar el modelo con los datos de entrenamiento
modelo_regresion.fit(X_train,Y_train)

LinearRegression()

### 5. Evaluación del modelo
Las métricas de evaluación nos van a permitir determinar qué tan bien se desempeña un modelo; es decir, cómo se ajusta a los datos. En esta sección se utilizan las siguientes métricas

1. Mean-Squared-Error(MSE). Error medio cuadrático
2. Mean-Absolute-Error(MAE). Error absoluto medio
3. R² or Coeficiente de determinación.

In [36]:
# Se obtienen las predicciones del modelo sobre el conjunto de entrenamiento.
y_pred = modelo_regresion.predict(X_train)
# Se obtienen las métricas a partir de la predicción y la base de evaluación (valores reales).
print("RMSE: %.2f" % mean_squared_error(Y_train, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_train, y_pred))
print('R²: %.2f' % r2_score(Y_train, y_pred))

RMSE: 331557.41
MAE: 179490.05
R²: 0.61


In [37]:
# Se obtienen las predicciones del modelo sobre el conjunto test.
y_pred = modelo_regresion.predict(X_test)
# Se obtienen las métricas a partir de la predicción y la base de evaluación (valores reales).
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 296420.56
MAE: 181448.78
R²: 0.64


Una vez determinado el rendimiento de generalización utilizando el conjunto test podemos tomar todos los datos y construir el modelo final.

In [38]:
# Ajustar el modelo con los datos de entrenamiento
modelo_regresion.fit(X,Y)

LinearRegression()

In [39]:
# Podemos visualizar los parámetros del modelos (coeficientes de regresión)
modelo_regresion.coef_

array([ 3.47808867e+04, -7.45643628e-01,  6.74865686e-01,  1.02812711e+04,
       -6.52778170e+03, -2.68394303e+04,  9.18335668e+03,  8.18682230e+04,
       -6.42121493e+04, -5.38259298e+05, -5.88979497e+05, -6.05227541e+05,
        2.34226702e+06, -6.09800685e+05])

### 6. Interpretación del modelo
También podemos sacar un reporte que nos va a permitir analizar estadísticos del modelo.

In [40]:
# Ajustar el modelo para ver el reporte
model = sm.OLS(Y, X).fit() ## sm.OLS(output, input)
# Mostrar las estadísticas del modelo
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 precio   R-squared:                       0.616
Model:                            OLS   Adj. R-squared:                  0.615
Method:                 Least Squares   F-statistic:                     894.2
Date:                Wed, 06 Sep 2023   Prob (F-statistic):               0.00
Time:                        21:53:23   Log-Likelihood:                -94758.
No. Observations:                6716   AIC:                         1.895e+05
Df Residuals:                    6703   BIC:                         1.896e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
modelo                            3.478e+04   1266.860     27.454      0.000    3.23e+04    3.73e+04
kilometraje                         -0.7456      0.078     -9.504      0.000      -0.899      -0.592
motor                                0.6749      0.409      1.649      0.099      -0.127       1.477
poder_maximo                      1.028e+04    135.797     75.711      0.000       1e+04    1.05e+04
asientos                         -6527.7817   3773.754     -1.730      0.084   -1.39e+04     869.977
combustible_CNG                  -3.877e+07   1.42e+06    -27.311      0.000   -4.16e+07    -3.6e+07
combustible_Diesel               -3.873e+07   1.42e+06    -27.330      0.000   -4.15e+07    -3.6e+07
combustible_LPG                  -3.866e+07   1.42e+06    -27.319      0.000   -4.14e+07   -3.59e+07
combustible_Petrol               -3.881e+07   1.42e+06    -27.404      0.000   -4.16e+07    -3.6e+07
propietario_First Owner          -3.153e+07   1.14e+06    -27.760      0.000   -3.38e+07   -2.93e+07
propietario_Fourth & Above Owner -3.158e+07   1.13e+06    -27.975      0.000   -3.38e+07   -2.94e+07
propietario_Second Owner          -3.16e+07   1.13e+06    -27.899      0.000   -3.38e+07   -2.94e+07
propietario_Test Drive Car       -2.865e+07   1.15e+06    -24.998      0.000   -3.09e+07   -2.64e+07
propietario_Third Owner           -3.16e+07   1.13e+06    -27.949      0.000   -3.38e+07   -2.94e+07
==============================================================================
Omnibus:                     7130.935   Durbin-Watson:                   1.698
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1038004.616
Skew:                           5.065   Prob(JB):                         0.00
Kurtosis:                      63.056   Cond. No.                     1.16e+21
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.4e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### 7. Almacenamiento  del modelo.
Puedes guardar tu modelo para que lo puedas recuperar cuando lo desees o llevarlo a producción.

In [41]:
# Una forma de guardar el modelo es a través de la funcionalidad joblib.
import joblib

In [29]:
# Ahora guardamos el modelo con el nombre que selecciones.
joblib.dump(modelo_regresion, 'ModeloRegresion.joblib')
# Si lo quieres recuperar más tarde puedes utilizar la siguiente instrucción:
# modelo = load('ModeloRegresion.joblib') 

['ModeloRegresion.joblib']